# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [139]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [140]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [141]:
box = driver.find_element_by_id("checkbox") 
box.click()
go = driver.find_element_by_xpath ('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
go.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [142]:
search = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [143]:
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [144]:
trade_name = driver.find_element_by_name('txtTradeName')

In [145]:
trade_name.send_keys ('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [146]:
table=driver.find_element_by_class_name ('searchtable')
table.submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [147]:
names=driver.find_elements_by_class_name ('searchlistitem')

In [148]:
for name in names:
    print (name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [149]:
results = driver.find_elements_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[3]/a/img')
for result in results:
    try:
        result.click()
    except:
        pass

Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [150]:
owners = driver.find_elements_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody')
for owner in owners:
    cell = owner.find_element_by_class_name ('tablecelltext')
    mailing_address = cell.text.split(" ")[:6]
    license_information = cell.text.split(" ")[6:]
    location_address = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
    total_paid = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
    issued_by = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
    ret_urn = driver.find_element_by_partial_link_text ('Return')
    ret_urn = result.click()

  
    

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [152]:
datapoints = []
fields = driver.find_elements_by_class_name ('searchfieldtitle')
names=driver.find_elements_by_class_name ('searchlistitem')
for field in fields:
    datapoint = {}
    name = field.find_element_by_class_name ('searchlistitem') 
    datapoint['name'] = name.text
    try:
        details = field.find_element_by_tag_name('a')
        datapoint['url'] = details.get_attribute('href')      
    except:
        pass
    datapoints.append(datapoint)
datapoints

[{'name': 'VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0E4%2B%2FMVouPw%3D'},
 {'name': 'VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=11t9IIRWJ0o%3D'},
 {'name': 'VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=UR1ep%2BTo1Nc%3D'},
 {'name': 'VAPE FROG'},
 {'name': 'VAPE FROG'}]

In [153]:
import pandas as pd

df = pd.DataFrame(datapoints)
df.head()

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0E4%2B%2FMVouPw%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=11t9IIRWJ0o%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=UR1ep%2BTo1Nc%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [154]:
more = driver.find_element_by_partial_link_text ('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", more)
more.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [155]:
while True:   
    datapoints = []
    fields = driver.find_elements_by_class_name ('searchfieldtitle')
    names=driver.find_elements_by_class_name ('searchlistitem')
    for field in fields:
        datapoint = {}
        name = field.find_element_by_class_name ('searchlistitem') 
        datapoint['name'] = name.text
        try:
            details = field.find_element_by_tag_name('a')
            datapoint['url'] = details.get_attribute('href')
        except:
            pass
        datapoints.append(datapoint)
    more = driver.find_element_by_partial_link_text ('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", more)
    more.click()
    print ("Scraping a new page")

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [156]:
try:
    while True:
        datapoints = []
        fields = driver.find_elements_by_class_name ('searchfieldtitle')
        names=driver.find_elements_by_class_name ('searchlistitem')
        for field in fields:
            datapoint = {}
            name = field.find_element_by_class_name ('searchlistitem') 
            datapoint['name'] = name.text
            try:
                details = field.find_element_by_tag_name('a')
                datapoint['url'] = details.get_attribute('href')
            except:
                pass
            datapoints.append(datapoint)
        more = driver.find_element_by_partial_link_text ('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", more)
        more.click()
        print ("Scraping a new page")
        df = pd.DataFrame(datapoints)
except:
    pass

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [157]:
df.to_csv("vape-shops-basic.csv", index=False)

In [132]:
df

,name,url
0,VAPOR VILLA,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Txs6wggPiGE%3D
1,VAPOR VILLA,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=jtgI0fjd5gk%3D
2,VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WopY7exl5tY%3D
3,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=G3sRcllREbM%3D
4,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=hWIfeJwWuZc%3D


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [158]:
pd.set_option('display.max_colwidth', -1)

In [159]:
df.url.head(1)

0    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=JHSfh89r1Tg%3D
Name: url, dtype: object

### Use Selenium to visit that page

In [160]:
for url in df['url']:
    driver.get(url)

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [161]:
owners = driver.find_elements_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody')
for owner in owners:
    cell = owner.find_element_by_class_name ('tablecelltext')
    mailing_address = cell.text.split(" ")[:6]
    license_information = cell.text.split(" ")[6:]
    location_address = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
    total_paid = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
    issued_by = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
   

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [185]:
for url in df['url']:
    driver.get(url)
    owners = driver.find_elements_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody')
    for owner in owners:
        owner_details = {}
        cell = owner.find_element_by_class_name ('tablecelltext')
        mailing_address = cell.text.split(" ")[:6]
        owner_details['mailing_address'] = mailing_address
        license_information = cell.text.split(" ")[6:]
        owner_details['license_information'] = license_information
        location_address = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
        owner_details['location_address'] = location_address
        try:
            total_paid = driver.find_element_by_name ('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]')
            owner_details['total_paid'] = total_paid
            issued_by = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
            owner_details['issued_by'] = issued_by
        except:
            pass

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [194]:
def get_vape_info(row):
    driver.get(row ['url'])
    owners = driver.find_elements_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody')
    for owner in owners:
        owner_details = {}
        cell = owner.find_element_by_class_name ('tablecelltext')
        mailing_address = cell.text.split(" ")[:6]
        license_information = cell.text.split(" ")[6:]
        location_address = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
        try:
            total_paid = driver.find_element_by_name ('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]')
            owner_details['total_paid'] = total_paid
            issued_by = driver.find_element_by_xpath ('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
            owner_details['issued_by'] = issued_by
        except:
            pass
    
    return pd.Series({
        'mailing_address': mailing_address,
        'license_information': license_information,
        'location_address': location_address,
        
})
    
    

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [200]:
new_df = df.apply(get_vape_info, axis=1).join(df)

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [201]:
new_df.to_csv("vape-total.csv", index=False)

In [202]:
new_df

,mailing_address,license_information,location_address,name,url
0,"[VAPOR, VILLA, CORP\nLicense, Status:, Issued\nLicense, No.:]","[03322784\nControl, No.:, 03696574\nDate, of, Issue:, 4/26/2018\nMonths, Paid:, 12\nExp., Date:, 4/30/2019\nSubdivision:, 03, Baltimore, County]","VAPOR VILLA\n1913 LANSDOWNE ROAD\nSUITE E\nBALTIMORE, MD 21227",VAPOR VILLA,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=JHSfh89r1Tg%3D
1,"[VAPOR, VILLA, CORP\nLicense, Status:, Issued\nLicense, No.:]","[03322785\nControl, No.:, 03696931\nDate, of, Issue:, 4/26/2018\nMonths, Paid:, 12\nExp., Date:, 4/30/2019\nSubdivision:, 03, Baltimore, County]","VAPOR VILLA\n730 FREDERICK ROAD\nSUITE 202\nCATONSVILLE, MD 21228",VAPOR VILLA,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ETVzKAWMMX8%3D
2,"[VAPORS, LOUNGE, LLC\nLicense, Status:, Issued\nLicense, No.:]","[18369505\nControl, No.:, 18877819\nDate, of, Issue:, 4/17/2018\nMonths, Paid:, 12\nExp., Date:, 4/30/2019\nSubdivision:, 18, St., Mary's, County]","VAPORS LOUNGE\n28943 THREE NOTCH ROAD\nMECHANICSVILLE, MD 20659",VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ruGgcs7dJ2g%3D
3,"[WALKER, TRADING, CO, INC\n11600, CROSSROADS, CIRCLE\nSUITE]","[E\nMIDDLE, RIVER,, MD, 21220, License, Status:, Issued\nLicense, No.:, 12341351\nControl, No.:, 12958340\nDate, of, Issue:, 5/29/2018\nMonths, Paid:, 12\nExp., Date:, 4/30/2019\nSubdivision:, 12, Bel, Air]","VAPE DOJO\n138 N BOND STREET\nBEL AIR, MD 21014",VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=LxA85lCh6pY%3D
4,"[WALKER, TRADING, COMPANY, INC\n11600, CROSSROADS, CIRCLE\nSUITE]","[E\nMIDDLE, RIVER,, MD, 21220, License, Status:, Issued\nLicense, No.:, 02310917\nControl, No.:, 02111914\nDate, of, Issue:, 6/01/2018\nMonths, Paid:, 12\nExp., Date:, 4/30/2019\nSubdivision:, 02, Anne, Arundel, County]","VAPE DOJO\n331 GAMBRILLS ROAD\nGAMBRILLS, MD 21054",VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=27K18vFmP2c%3D
